### Audiobooks project

### In this version the data is shuffled before balancing it. 

In [13]:
# Importing libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [14]:
# Loading the data
raw_csv_data = np.loadtxt('data/Audiobooks_data.csv', delimiter = ",")
raw_csv_data

array([[9.9400e+02, 1.6200e+03, 1.6200e+03, ..., 5.0000e+00, 9.2000e+01,
        0.0000e+00],
       [1.1430e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0590e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 3.8800e+02,
        0.0000e+00],
       ...,
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00],
       [2.5100e+02, 1.6740e+03, 3.3480e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [15]:
raw_d = pd.read_csv('data/Audiobooks_data.csv')
raw_d.describe()

,00994,1620,1620.1,19.73,19.73.1,1,10.00,0.99,1603.80,5,92,0
count,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000
mean,16773.611943,1591.279646,1678.612796,7.102894,7.542940,0.160690,8.909717,0.125598,189.788585,0.069871,61.932898,0.158844
std,9691.239210,504.358512,654.861664,4.930700,5.559378,0.367258,0.643363,0.241104,370.905846,0.470342,88.210402,0.365544
min,2.000000,216.000000,216.000000,3.860000,3.860000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8371.500000,1188.000000,1188.000000,5.330000,5.330000,0.000000,8.910000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,16715.000000,1620.000000,1620.000000,5.950000,6.070000,0.000000,8.910000,0.000000,0.000000,0.000000,11.000000,0.000000
75%,25187.500000,2160.000000,2160.000000,8.000000,8.000000,0.000000,8.910000,0.130000,194.400000,0.000000,105.000000,0.000000
max,33683.000000,2160.000000,7020.000000,130.940000,130.940000,1.000000,10.000000,1.000000,2160.000000,30.000000,464.000000,1.000000


In [16]:
# In the firs column there is ID (we don't use it) and the last column are targets
# We have to split this data:
unscaled_inputs_all = raw_csv_data[:,1:-1] #all the rows, columns from 1 to -1 (excluded)
targets_all = raw_csv_data[:,-1] #all the rows, only last column

### Shuffling indices

In [17]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

#### The data is shuffled randomly so every time we reuse this notebook the data will be shuffled in different way. There is no random seed in here because this is the very basic and simple approach for EDA. 

### Balancing the dataset


In [18]:
# Counting 1's in the targets
num_one_targets  = int(np.sum(targets_all))
num_one_targets

2237

#### In this course the approach for balancing the dataset is to leave only the same amount of targets with 0 value as we have num_one_targets. The rest is going to be deleted.

In [19]:
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter +=1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

In [20]:
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardizing

In [21]:
# Scaling using sklearn library (standardizing data along any axis)
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffling the data

### Even though we shuffled the data at the beginning we still have to shuffle it after balancing the dataset. This is because otherwise we would have all "1" targets in the training set.

In [22]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Splitting the data

In [23]:
# Getting the sizes of train, test and validations sets
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count-train_samples_count-validation_samples_count

# Splitting the data (manually)
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [24]:
# We can check if the splitted data is balanced as we intended:
print(np.sum(train_targets), train_samples_count, np.sum(train_targets/train_samples_count))
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets/validation_samples_count))
print(np.sum(test_targets), test_samples_count, np.sum(test_targets/test_samples_count))

1796.0 3579 0.5018161497625035
223.0 447 0.49888143176733774
218.0 448 0.4866071428571428


#### The second columns is the number of samples which for the first row (training set) should be much bigger than for the last two (val and test). In the last column all values should be around 0.5 (50%) which shows us the split. In this case everything is like it should be.

### Saving the data

In [25]:
# We can save the data in the zipped numpy file format (.npz) for future use
np.savez('data/audiobooks_data_train_v1', inputs=train_inputs, targets=train_targets)
np.savez('data/audiobooks_data_validation_v1', inputs=validation_inputs, targets = validation_targets)
np.savez('data/audiobooks_data_test_v1', inputs=test_inputs, targets=test_targets)